# Data Combination over Movie Data Sets (CSV Files)

The purpose of this notebook is to document the steps taken in order to generate movies_df and studios_df, which will be used for further analysis to make recommendations to Microsoft on where they could enter the original movie production business.

The scope of this excercise is the provided CSV Files:
- bom.Movie_gross.csv.gz
- rt.movie_info_tsv.gz
- rt.reviews.tsv.gz
- tmdb.movies.csv.gz
- tn.movie_budgets.csv.gz

<figure>
    <img src="Converging Arrows.png"
         alt="Let's synthesize some data"
         width="400"
         height="200">
    <figcaption><center><bold>Putting it all together</figcaption>
</figure>

From the EDA notebook:

The CSV files will contain the relevant financial data needed to make a recommendation to Microsoft. Our key metric will be profitability of movie titles, then profitability by genre, and then identifying people in the movie making labor force would be best for the job (using a mix of both movie profitability and critic reviews)

## Master Table Import: "tn.movie_budgets.csv.gz"

### *Basic Import*

For purposes of our combination, we will use 'tn.movie_budgets.csv.gz' as our beginning source of truth, as it contains the most relevant information we are looking to investigate (mainly, domestic and international box office sales and movie budget)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import string
punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'

In [2]:
df_budgets = pd.read_csv("Raw Data/tn.movie_budgets.csv.gz")

In [3]:
df_budgets.head(2)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


Converting the financial data into numbers, release_date info into datetime objects, and creating a 'year' column:

In [4]:
df_budgets['worldwide_gross'] = df_budgets['worldwide_gross'].str.replace('$', '').str.replace(',', '')
df_budgets['domestic_gross'] = df_budgets['domestic_gross'].str.replace('$', '').str.replace(',', '')
df_budgets['production_budget'] = df_budgets['production_budget'].str.replace('$', '').str.replace(',', '')

df_budgets['worldwide_gross'] = df_budgets['worldwide_gross'].astype(float)
df_budgets['domestic_gross'] = df_budgets['domestic_gross'].astype(float)
df_budgets['production_budget'] = df_budgets['production_budget'].astype(float)

df_budgets['release_date'] = pd.to_datetime(df_budgets['release_date'])

df_budgets['year'] = df_budgets['release_date'].dt.strftime('%Y')
df_budgets['year'] = df_budgets['year'].map(int)

Since we will be combining additional genre info based off of the title of the movie, we will normalize punctuation use by removing punctuation from all titles in this data source and later data sources:

In [5]:
transtab = str.maketrans(dict.fromkeys(punct, ''))

df_budgets['movie'] = '|'.join(df_budgets['movie'].tolist()).translate(transtab).split('|')

We will create a profit calculation, which is worldwide_gross minus the production budget. We will also create a column which calculates the percentage of profit margin

In [6]:
df_budgets['profit'] = df_budgets['worldwide_gross'] - df_budgets['production_budget']
df_budgets['profit_margin'] = (df_budgets['profit'] / df_budgets['worldwide_gross']) * 100

In [7]:
df_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,profit,profit_margin
0,1,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2009,2.351345e+09,84.692106
1,2,2011-05-20,Pirates of the Caribbean On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,6.350639e+08,60.733080
2,3,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019,-2.002376e+08,-133.703598
3,4,2015-05-01,Avengers Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,1.072414e+09,76.436443
4,5,2017-12-15,Star Wars Ep VIII The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,9.997217e+08,75.925058


## Master Table Import Filtering

Filter Criteria:

A studio like Microsoft will likely put at least 500k into it's projects, so we will filter for budgets above 500k. Since this is a new studio, any individual movie above 100m might be too risky. 

They will likely not be interested in festivals or other distributions that will not have box office values, so we will filter out movies with zero box office sales. Additionally, it is unlikely that this movie would completely flop, so lets rule out sales of less than $50k. 

We also are only interested in more modern data, which we will define as anything that was produced in the past 15 years

In [8]:
df_budgets = df_budgets[(df_budgets['production_budget'] > 500000) & (df_budgets['production_budget'] < 100000000)]
df_budgets = df_budgets[(df_budgets['worldwide_gross'] > 50000)]
df_budgets = df_budgets[df_budgets['year'] > 2008]

We will need to join this table with other tables, however there are duplicate values in this data set as multiple movies can share the same title. For purposes of joining, we will join on a new column, movie_and_year, created below:

In [9]:
df_budgets['year'] = df_budgets['year'].map(str)
df_budgets['movie_and_year'] = df_budgets['movie'] + " - " + df_budgets['year']
df_budgets['year'] = df_budgets['year'].map(int)

In [10]:
df_budgets[df_budgets.duplicated(['movie_and_year'], keep=False)]

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,profit,profit_margin,movie_and_year


The above confirms no duplicate values

## Appending Genre using additional IMDB Data

The SQL information which contains movie title and genres is insufficient at appending associate genres with a movie, so we will import additional data found on the IMBD website (https://datasets.imdbws.com/). These files are rather large so they will be ignored in git pushes

In [11]:
df_additional = pd.read_csv("Raw Data/title.basics.tsv.gz", sep='\t', encoding='latin1')
df_additional['primaryTitle'] = df_additional['primaryTitle'].map(str)
df_additional = df_additional[df_additional['titleType'] == 'movie']

C:\Users\moore\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
transtab = str.maketrans(dict.fromkeys(punct, ''))

df_additional['primaryTitle'] = '|'.join(df_additional['primaryTitle'].tolist()).translate(transtab).split('|')

Appending a movie_and_year column:

In [13]:
df_additional['startYear'] = pd.to_datetime(df_additional['startYear'], errors='coerce')
df_additional['year'] = df_additional['startYear'].dt.strftime('%Y')

df_additional['movie_and_year'] = df_additional['primaryTitle'] + " - " + df_additional['year']

df_additional = df_additional.fillna(0)
df_additional['year'] = df_additional['year'].map(int)

## Appending Genre using CSV Data

In [14]:
df_budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1722 entries, 406 to 5399
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1722 non-null   int64         
 1   release_date       1722 non-null   datetime64[ns]
 2   movie              1722 non-null   object        
 3   production_budget  1722 non-null   float64       
 4   domestic_gross     1722 non-null   float64       
 5   worldwide_gross    1722 non-null   float64       
 6   year               1722 non-null   int64         
 7   profit             1722 non-null   float64       
 8   profit_margin      1722 non-null   float64       
 9   movie_and_year     1722 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(2), object(2)
memory usage: 148.0+ KB


In [15]:
df_budget_genre = df_budgets.merge(df_additional, how='left', left_on=['movie_and_year'], right_on=['movie_and_year'])

In [16]:
df_budget_genre.isna().sum()

id                     0
release_date           0
movie                  0
production_budget      0
domestic_gross         0
worldwide_gross        0
year_x                 0
profit                 0
profit_margin          0
movie_and_year         0
tconst               500
titleType            500
primaryTitle         500
originalTitle        500
isAdult              500
startYear            500
endYear              500
runtimeMinutes       500
genres               500
year_y               500
dtype: int64

In [17]:
df_budget_genre = df_budget_genre.dropna()

In [18]:
df_budget_genre.drop_duplicates(subset="movie_and_year",keep=False, inplace=True)

In [19]:
df_budget_genre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1179 entries, 0 to 1772
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1179 non-null   int64         
 1   release_date       1179 non-null   datetime64[ns]
 2   movie              1179 non-null   object        
 3   production_budget  1179 non-null   float64       
 4   domestic_gross     1179 non-null   float64       
 5   worldwide_gross    1179 non-null   float64       
 6   year_x             1179 non-null   int64         
 7   profit             1179 non-null   float64       
 8   profit_margin      1179 non-null   float64       
 9   movie_and_year     1179 non-null   object        
 10  tconst             1179 non-null   object        
 11  titleType          1179 non-null   object        
 12  primaryTitle       1179 non-null   object        
 13  originalTitle      1179 non-null   object        
 14  isAdult 

In [20]:
from pathlib import Path  
filepath = Path('financial_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_budget_genre.to_csv(filepath)  